<a href="https://colab.research.google.com/github/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decoding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-16 04:41:19--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-16 04:41:20--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-16 04:41:20--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,FC3,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value
0,0,136,1,1,-200,-2.6587,-5.8283,-3.6200,-8.4912,-6.0398,...,-4.2465,-6.1961,-0.7449,-4.0477,-6.5453,-252.5419,70.5294,13.6076,-66.9374,1.0
1,1,136,1,1,-196,-3.3376,-5.6611,-3.5900,-8.8003,-6.9963,...,-3.2880,-5.5053,-0.7434,-3.1528,-4.8430,-218.5595,51.3878,25.2795,-68.4394,1.0
2,2,136,1,1,-192,-3.4338,-4.6322,-2.9857,-8.3836,-7.6261,...,-2.0604,-4.9363,-0.8046,-2.6793,-2.9363,-196.0244,68.5147,18.5178,-64.9081,1.0
3,3,136,1,1,-188,-3.6411,-3.6174,-2.5697,-7.9997,-8.4763,...,-1.1900,-5.0391,-0.8101,-2.7765,-1.7893,-191.9580,51.4932,22.3986,-66.2655,1.0
4,4,136,1,1,-184,-3.9098,-3.0866,-2.5453,-8.4382,-9.8238,...,-0.9082,-5.5007,-0.6975,-3.0212,-2.2171,-177.3466,41.4210,21.8185,-66.5214,1.0


In [0]:
#df_one_hot_encoded = pd.get_dummies(df)
#df = df_one_hot_encoded


In [4]:
df_1 = df.groupby(np.arange(len(df))//5).mean()
df_1['High Value'] = 0
df_1.loc[df_1['Value'] >5, 'High Value'] = 1
df_1.head(60)

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,F3,FP2,F7,FP1,VEOU,VEOL,HEOR,HEOL,Value,High Value
0,2,136,1,1,-192,-3.39620,-4.56512,-3.06214,-8.42260,-7.79246,...,-5.43550,-0.76010,-3.13550,-3.66620,-207.28608,56.66922,20.32440,-66.61436,1.0,0
1,7,136,1,1,-172,-5.20628,-0.32864,-2.77046,-9.07170,-12.00468,...,-2.41326,-1.93568,-2.28162,-3.04326,-106.94820,1.75514,25.80194,-56.04718,1.0,0
2,12,136,1,1,-152,-2.84608,5.15314,0.78680,-5.60850,-7.75482,...,-4.15404,-3.17186,-2.11862,-4.03626,68.62368,-68.33450,20.30014,-18.55150,1.0,0
3,17,136,1,1,-132,8.96336,4.41986,6.62588,5.94982,9.95042,...,4.52602,7.27456,1.10208,1.26568,165.87524,-90.30248,18.97206,16.57666,1.0,0
4,22,136,1,1,-112,4.36154,-2.21088,1.13674,5.53136,6.81236,...,5.28440,6.59148,4.29536,8.84988,182.50180,-86.96050,9.76776,32.49734,1.0,0
5,27,136,1,1,-92,1.79962,4.60384,3.35576,6.10372,1.47634,...,-0.28368,-1.62946,2.39350,0.10214,131.59946,-47.10480,-12.50178,38.19382,1.0,0
6,32,136,1,1,-72,7.31702,3.71566,5.81652,7.83620,10.13494,...,-6.38296,-5.15328,-0.88880,-0.77766,45.96038,7.06100,-16.65158,29.75986,1.0,0
7,37,136,1,1,-52,3.54224,-4.16302,-0.50312,6.08560,10.01846,...,-2.77510,-2.73240,-1.33546,0.35156,-42.48102,52.07448,-17.35840,15.17392,1.0,0
8,42,136,1,1,-32,-4.62888,-0.31314,-3.19088,-1.04696,-2.00892,...,5.18670,-0.27704,1.63624,1.21138,-102.86696,75.15104,-26.32326,6.31374,1.0,0
9,47,136,1,1,-12,-9.90632,-6.31166,-8.19508,-7.35694,-8.83172,...,6.44744,1.79376,0.33278,-0.25722,-134.97826,99.99146,-22.33126,2.69774,1.0,0


In [5]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[185, 47, 45, 119, 7, 126, 72, 97, 174, 80, 18, 37, 190, 129, 63, 177, 105, 149, 104, 71, 40, 189, 167, 79, 86, 186, 51, 108, 123, 113, 92, 62, 133, 118, 183, 66, 90, 145, 39, 101, 170, 122, 77, 6, 120, 69, 15, 20, 42, 95, 36, 132, 81, 181, 56, 158, 67, 33, 143, 52, 59, 175, 153, 2, 112, 197, 49, 148, 60, 173, 43, 188, 195, 1, 30, 144, 64, 73, 29, 31, 137, 55, 82, 8, 28, 48, 19, 87, 70, 196, 116, 135, 83, 21, 17, 89, 124, 65, 150, 178]
[3, 4, 5, 9, 10, 11, 12, 13, 14, 16, 22, 23, 24, 25, 26, 27, 32, 34, 35, 38, 41, 44, 46, 50, 53, 54, 57, 58, 61, 68, 74, 75, 76, 78, 84, 85, 88, 91, 93, 94, 96, 98, 99, 100, 102, 103, 106, 107, 109, 110, 111, 114, 115, 117, 121, 125, 127, 128, 130, 131, 134, 136, 138, 139, 140, 141, 142, 146, 147, 151, 152, 154, 155, 156, 157, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 171, 172, 176, 179, 180, 182, 184, 187, 191, 192, 193, 194, 198, 199, 200]


In [0]:
rfc = RandomForestClassifier(n_estimators=500, max_features='auto')
#rfc.fit(x_pd, y_pd);

In [0]:
#df_1 = df_1.drop(['Unnamed: 0','Subject'], axis=1)
df_2.head()

,Unnamed: 0,Subject,Block,Trial,Time,O2,O1,OZ,PZ,P4,...,F8,T7,FT7,FC3,F3,FP2,F7,FP1,HEOL,Value
30,152,136,1,1,408,-15.67550,2.58324,-9.76768,-12.61532,-14.14894,...,-11.54476,1.64140,-5.65690,-6.24994,-7.14656,-11.86244,-6.25888,-5.47406,-32.73586,1.0
31,157,136,1,1,428,-15.82038,2.46982,-9.67022,-15.54158,-16.23420,...,-10.42348,-7.42732,-8.99196,-14.03126,-9.91762,-7.47934,-9.43366,-6.11862,-28.45538,1.0
32,162,136,1,1,448,-14.78484,3.02954,-7.90768,-12.48664,-13.88312,...,-15.01556,-8.61574,-12.11504,-10.19044,-10.78694,-13.19982,-12.44140,-7.38176,-30.48704,1.0
33,167,136,1,1,468,-16.44966,4.57424,-7.86714,-11.99818,-17.26026,...,-14.85436,-2.81998,-7.82412,-6.75134,-12.32264,-14.42776,-8.17064,-5.40640,-29.37572,1.0
34,172,136,1,1,488,-11.28078,7.24516,-4.64198,-12.59942,-13.38544,...,-9.82124,-6.94452,-6.28568,-10.68608,-11.07042,-11.30868,-6.64954,-10.02242,-31.81244,1.0


In [7]:
from sklearn.model_selection import GroupShuffleSplit
n_splits = 20
cv = GroupShuffleSplit(n_splits=n_splits, test_size=0.2)
#df_2 = df_1[(df_1['Value']==1) | (df_1['Value']==9)]
#df_2 = df_2.drop(['O1','O2','OZ','FP1','FP2','F3','FZ','F4','P8','TP8','P7','FC4','FT8','TP7','FCZ','F8','F7','FT7','VEOU','VEOL','HEOR','HEOR'], axis=1)
df_2 = df_1.drop(['VEOU','VEOL','HEOR','HEOR'], axis=1)
#df_2 = df_2[df_2['Time'] < 800]
#df_2 = df_2[df_2['Time'] > 400]
r_in = np.zeros(n_splits)
r_out = np.zeros(n_splits)
for i,(train, test) in enumerate(cv.split(df_2, groups=df_2['Trial'])):
  #train = df_2.index[train]
  #test = df_2.index[test]
  y = df_2['High Value']
  X = df_2.drop('High Value', axis=1).astype('float')
  rfc.fit(X.iloc[train].values, y.iloc[train])
  predicted = rfc.predict(X.iloc[train])
  actual = y.iloc[train]
  r_in[i] = np.corrcoef(predicted, actual)[0, 1]
  t_predicted = rfc.predict(X.iloc[test])
  t_actual = y.iloc[test]
  r_out[i] = np.corrcoef(t_predicted, t_actual)[0, 1]
  print(r_in[i], r_out[i])
print("Mean R_out for entire time series = %.3g +/- %.3g" % (r_out.mean(), r_out.std()))


1.0 0.9999999999999999
1.0 1.0
1.0 1.0
1.0 0.9999999999999999
1.0 1.0
1.0 1.0
1.0 1.0
1.0 0.9999999999999999
1.0 1.0
1.0 1.0
1.0 1.0
1.0 0.9999999999999999
1.0 1.0
1.0 0.9999999999999999
0.9999999999999999 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.9999999999999999 1.0
1.0 0.9999999999999999
Mean R_out for entire time series = 1 +/- 6.08e-17


In [0]:
plt.scatter(t_predicted,predicted)

In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [0]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
